In [2]:
import numpy as np
import torch 
import torch.nn as nn
from tqdm import tnrange
import torch.optim as optim
import torch.nn.functional as F
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.metrics import explained_variance_score
import scipy
import time

from rslds import RSLDS

In [4]:
print(1 / np.logspace(-0.5,-3,50))

[   3.16227766    3.55648031    3.9998234     4.49843267    5.05919749
    5.68986603    6.39915234    7.19685673    8.09400122    9.10298178
   10.23773966   11.51395399   12.94925842   14.56348478   16.37893707
   18.42069969   20.716984     23.29951811   26.20398529   29.47051703
   33.14424749   37.2759372    41.92267435   47.14866363   53.02611336
   59.63623317   67.07035611   75.43120063   84.83428982   95.40954763
  107.30309405  120.67926406  135.7228783   152.64179672  171.66979066
  193.06977289  217.13743029  244.20530945  274.64741148  308.88435965
  347.38921121  390.69399371  439.39705608  494.17133613  555.77365865
  625.05519253  702.97321153  790.60432109  889.15933392 1000.        ]


In [2]:
base_dir = "/media/hdd01/sklee/"
experiment = "clust4-60"
cell_type = "CA1"
E_neural_file = "Espikes_neural.npz"
V_file = "V_diff.npy"
eloc_file = "Elocs_T10_Ne2000_gA0.6_tauA1_gN0.8_Ni200_gG0.1_gB0.1_Er0.5_Ir7.4_random_NR_rep1000_stimseed1.npy"

E_neural = scipy.sparse.load_npz(base_dir+cell_type+"_"+experiment+"/data/"+E_neural_file)
V = np.load(base_dir+cell_type+"_"+experiment+"/data/"+V_file)
V = torch.from_numpy(V)
eloc = np.load(base_dir+cell_type+"_"+experiment+"/data/"+eloc_file)

den_idx = np.unique(eloc[880:1120,0])
e_idx = np.where(np.isin(eloc[:,0], den_idx) == True)[0]
e_idx = torch.from_numpy(e_idx)

In [3]:
T_train = 999 * 1000 * 50
T_test = 1 * 1000 * 50
hid_no = 2 # H
sub_no = 4 # N
state_no = 3 # K
in_no = 299
save_dir = base_dir+cell_type+"_"+experiment+"/"
device = torch.device("cuda")

batch_length = 50000
batch_size = 9
iter_no = 10000
epoch_no = iter_no*batch_length*batch_size//T_train 

In [4]:
V_train = V[:T_train].float()
V_test = V[T_train:T_train + T_test].to(device).float()

test_E_neural = E_neural[T_train:T_train+T_test].toarray()
train_E_neural = E_neural[:T_train]
test_E_neural = torch.from_numpy(test_E_neural).float().to(device)

train_idx = np.empty((epoch_no, T_train//batch_length//batch_size))
for i in range(epoch_no):
    part_idx = np.arange(0, T_train, batch_length*batch_size)
    np.random.shuffle(part_idx)
    train_idx[i] = part_idx
train_idx = train_idx.flatten()
train_idx = torch.from_numpy(train_idx)

In [5]:
C_syn = torch.zeros(sub_no, in_no).to(device)
for i in range(in_no):
    idx = e_idx[i]
    if eloc[idx,0] == den_idx[0]:
        C_syn[0,i] = 1
    elif eloc[idx,0] == den_idx[1]:
        C_syn[1,i] = 1
    elif eloc[idx,0] == den_idx[2]:
        C_syn[2,i] = 1
    elif eloc[idx,0] == den_idx[3]:
        C_syn[3,i] = 1

In [6]:
model = RSLDS(batch_length, sub_no, state_no, hid_no, batch_size, C_syn, device)
model.to(device).float()
print(sum(p.numel() for p in model.parameters() if p.requires_grad))

3600555


In [7]:
temp_list = np.logspace(-0.5, -3, 50)
temp_count = 0

for i in tnrange(iter_no):
    s = time.time()
    model.train()
    
    if (i%50 == 49) & (temp_count < 49):
        temp_count += 1
    temp = temp_list[temp_count] 
    
    batch_idx = train_idx[i].long()
    batch_E_neural = train_E_neural[batch_idx : batch_idx+batch_length*batch_size].toarray().reshape(batch_size, batch_length, -1)
    batch_E_neural = torch.from_numpy(batch_E_neural).float().to(device)
    batch_V = V_train[batch_idx : batch_idx+batch_length*batch_size].reshape(batch_size, -1).to(device)
    logprob_z1 = torch.log(torch.ones(state_no).to(device) * 1/state_no)
    
    model.X_inf.data = torch.zeros(batch_length, batch_length, sub_no, hid_no).to(device)
    
    # Calculate P(Z) and related likelihoods and posterior marginals
    log_A, log_B = model.calculate_likelihoods_for_z(batch_E_neural[:,:,e_idx], temp)
    forward_probs, backward_probs, log_gamma1, log_gamma2 = model.forward_backward(log_A, log_B, logprob_z1)
    
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
    for j in tnrange(50):
        # Optimize X and Theta
        optimizer.zero_grad()
        loss = model.calculate_likelihoods_for_x_theta(log_gamma1, log_gamma2, batch_E_neural, batch_V, temp)
        loss.backward()
        optimizer.step()
    
        print(i, loss)
    print(time.time() - s)

<ipython-input-7-01da07e9a52b>:4: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for i in tnrange(iter_no):


RuntimeError: [enforce fail at CPUAllocator.cpp:65] . DefaultCPUAllocator: can't allocate memory: you tried to allocate 80000000000 bytes. Error code 12 (Cannot allocate memory)